# 📊 Volume & Demand Analysis Dashboard

**Purpose**: Analyze order volumes, trends over time, and demand patterns by various dimensions.

## Key Metrics
- Order volume trends (daily, weekly, monthly)
- Volume by Product, Customer, Domain
- Month-over-month growth
- Demand patterns and forecasting

---

## 1. Setup & Data Loading

In [ ]:
# Install required packages
!pip install pandas openpyxl plotly seaborn matplotlib -q

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

pd.set_option('display.max_columns', None)
print("✅ Libraries loaded successfully!")

In [ ]:
# File path - update this to match your file location
filename = r"C:\Users\bmalaraju\Documents\WP-OP Agent\JIRA-Agent\11.25.WP Orders_25-11-2025_v01.xlsx"
print(f"📁 Using file: {filename}")

In [ ]:
# Load data
df = pd.read_excel(filename, engine='openpyxl')
print(f"📊 Dataset loaded: {len(df):,} rows, {len(df.columns)} columns")
print(f"📅 Analysis Date: {datetime.now().strftime('%Y-%m-%d %H:%M')}")

## 2. Data Preparation

In [ ]:
# Column mapping
COLUMN_MAP = {
    'product': 'Product',
    'customer': 'Customer',
    'domain': 'Domain',
    'market': 'Market',
    'project_name': 'Project Name',
    'order_id': 'WP Order ID',
    'quantity': 'WP Quantity',
    'added_date': 'Added Date',
    'status': 'WP Order Status',
    'sow_pa': 'SOW/PA',
    'po_number': 'PO Number'
}

# Validate columns
missing = [v for v in COLUMN_MAP.values() if v not in df.columns]
if missing:
    print(f"⚠️ Missing columns (will use defaults): {missing}")
print("✅ Column mapping configured!")

In [ ]:
# Prepare analysis dataframe
analysis_df = df.copy()

# Parse dates
added_col = COLUMN_MAP['added_date']
if added_col in analysis_df.columns:
    analysis_df['added_date'] = pd.to_datetime(analysis_df[added_col], errors='coerce')
else:
    analysis_df['added_date'] = pd.NaT

# Parse quantities
qty_col = COLUMN_MAP['quantity']
if qty_col in analysis_df.columns:
    analysis_df['quantity'] = pd.to_numeric(analysis_df[qty_col], errors='coerce').fillna(0).astype(int)
else:
    analysis_df['quantity'] = 1

# Add time dimensions
analysis_df['year'] = analysis_df['added_date'].dt.year
analysis_df['month'] = analysis_df['added_date'].dt.month
analysis_df['week'] = analysis_df['added_date'].dt.isocalendar().week
analysis_df['year_month'] = analysis_df['added_date'].dt.to_period('M')
analysis_df['year_week'] = analysis_df['added_date'].dt.to_period('W')
analysis_df['day_of_week'] = analysis_df['added_date'].dt.day_name()

print(f"✅ Data prepared!")
print(f"📅 Date range: {analysis_df['added_date'].min()} to {analysis_df['added_date'].max()}")

## 3. Overall Volume Summary

In [ ]:
# Calculate overall metrics
total_orders = len(analysis_df)
unique_orders = analysis_df[COLUMN_MAP['order_id']].nunique() if COLUMN_MAP['order_id'] in analysis_df.columns else total_orders
total_quantity = analysis_df['quantity'].sum()
unique_products = analysis_df[COLUMN_MAP['product']].nunique() if COLUMN_MAP['product'] in analysis_df.columns else 0
unique_customers = analysis_df[COLUMN_MAP['customer']].nunique() if COLUMN_MAP['customer'] in analysis_df.columns else 0

# Average metrics
avg_qty_per_order = total_quantity / total_orders if total_orders > 0 else 0

# Display summary
print("="*60)
print("📊 VOLUME SUMMARY")
print("="*60)
print(f"\n📋 Total Rows:          {total_orders:,}")
print(f"📦 Unique Orders:       {unique_orders:,}")
print(f"📈 Total Quantity:      {total_quantity:,}")
print(f"📊 Avg Qty/Order:       {avg_qty_per_order:.1f}")
print(f"\n🏷️ Unique Products:     {unique_products:,}")
print(f"👥 Unique Customers:    {unique_customers:,}")
print("="*60)

In [ ]:
# KPI Cards visualization
fig = make_subplots(
    rows=1, cols=4,
    specs=[[{'type': 'indicator'}]*4],
    subplot_titles=['Total Orders', 'Total Quantity', 'Products', 'Customers']
)

fig.add_trace(go.Indicator(
    mode="number",
    value=total_orders,
    number={'font': {'size': 40, 'color': '#3498DB'}}
), row=1, col=1)

fig.add_trace(go.Indicator(
    mode="number",
    value=total_quantity,
    number={'font': {'size': 40, 'color': '#2ECC71'}}
), row=1, col=2)

fig.add_trace(go.Indicator(
    mode="number",
    value=unique_products,
    number={'font': {'size': 40, 'color': '#E67E22'}}
), row=1, col=3)

fig.add_trace(go.Indicator(
    mode="number",
    value=unique_customers,
    number={'font': {'size': 40, 'color': '#9B59B6'}}
), row=1, col=4)

fig.update_layout(height=200, title={'text': 'Volume KPIs', 'x': 0.5})
fig.show()

## 4. Volume Trends Over Time

In [ ]:
# Weekly volume trend
weekly_df = analysis_df[analysis_df['added_date'].notna()].copy()
weekly_df['week_start'] = weekly_df['added_date'].dt.to_period('W').dt.start_time

weekly_volume = weekly_df.groupby('week_start').agg(
    orders=('quantity', 'count'),
    quantity=('quantity', 'sum')
).reset_index()

# Show last 12 weeks
weekly_volume = weekly_volume.tail(12)

fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Bar(x=weekly_volume['week_start'], y=weekly_volume['orders'], name='Orders',
           marker_color='#3498DB', opacity=0.7),
    secondary_y=False
)

fig.add_trace(
    go.Scatter(x=weekly_volume['week_start'], y=weekly_volume['quantity'], name='Quantity',
               mode='lines+markers', line=dict(color='#E67E22', width=3)),
    secondary_y=True
)

fig.update_layout(
    title={'text': 'Weekly Order Volume Trend', 'x': 0.5, 'font': {'size': 20}},
    xaxis_title='Week',
    height=400,
    legend={'orientation': 'h', 'y': 1.1}
)

fig.update_yaxes(title_text="Order Count", secondary_y=False)
fig.update_yaxes(title_text="Total Quantity", secondary_y=True)

fig.show()

In [ ]:
# Monthly volume trend with growth rate
monthly_df = analysis_df[analysis_df['added_date'].notna()].copy()
monthly_df['month_start'] = monthly_df['added_date'].dt.to_period('M').dt.start_time

monthly_volume = monthly_df.groupby('month_start').agg(
    orders=('quantity', 'count'),
    quantity=('quantity', 'sum')
).reset_index()

# Calculate MoM growth
monthly_volume['mom_growth'] = monthly_volume['orders'].pct_change() * 100

fig = make_subplots(specs=[[{"secondary_y": True}]])

fig.add_trace(
    go.Bar(x=monthly_volume['month_start'], y=monthly_volume['orders'], name='Orders',
           marker_color='#2ECC71'),
    secondary_y=False
)

fig.add_trace(
    go.Scatter(x=monthly_volume['month_start'], y=monthly_volume['mom_growth'], name='MoM Growth %',
               mode='lines+markers', line=dict(color='#E74C3C', width=2, dash='dash')),
    secondary_y=True
)

fig.update_layout(
    title={'text': 'Monthly Order Volume & Growth', 'x': 0.5, 'font': {'size': 20}},
    xaxis_title='Month',
    height=400,
    legend={'orientation': 'h', 'y': 1.1}
)

fig.update_yaxes(title_text="Order Count", secondary_y=False)
fig.update_yaxes(title_text="MoM Growth %", secondary_y=True)

# Add zero line for growth
fig.add_hline(y=0, line_dash="dash", line_color="gray", secondary_y=True)

fig.show()

## 5. Volume by Product

In [ ]:
# Top products by order volume
product_col = COLUMN_MAP['product']

if product_col in analysis_df.columns:
    product_volume = analysis_df.groupby(product_col).agg(
        orders=('quantity', 'count'),
        quantity=('quantity', 'sum')
    ).reset_index()
    product_volume = product_volume.sort_values('orders', ascending=False)
    
    # Top 15 products
    top_products = product_volume.head(15)
    
    fig = go.Figure(data=[go.Bar(
        y=top_products[product_col],
        x=top_products['orders'],
        orientation='h',
        marker_color='#3498DB',
        text=top_products['orders'],
        textposition='outside'
    )])
    
    fig.update_layout(
        title={'text': 'Top 15 Products by Order Volume', 'x': 0.5, 'font': {'size': 20}},
        xaxis_title='Number of Orders',
        yaxis_title='Product',
        yaxis={'categoryorder': 'total ascending'},
        height=500,
        margin={'l': 200}
    )
    
    fig.show()
    
    # Product concentration
    print(f"\n📊 Product Concentration:")
    print(f"   Top 5 products: {product_volume.head(5)['orders'].sum() / total_orders * 100:.1f}% of orders")
    print(f"   Top 10 products: {product_volume.head(10)['orders'].sum() / total_orders * 100:.1f}% of orders")

In [ ]:
# Product volume treemap
if product_col in analysis_df.columns:
    # Use top 20 for treemap
    treemap_data = product_volume.head(20).copy()
    
    fig = px.treemap(
        treemap_data,
        path=[product_col],
        values='orders',
        color='quantity',
        color_continuous_scale='Blues',
        title='Product Distribution (Size = Orders, Color = Quantity)'
    )
    
    fig.update_layout(height=500)
    fig.show()

## 6. Volume by Customer

In [ ]:
# Top customers by order volume
customer_col = COLUMN_MAP['customer']

if customer_col in analysis_df.columns:
    customer_volume = analysis_df.groupby(customer_col).agg(
        orders=('quantity', 'count'),
        quantity=('quantity', 'sum')
    ).reset_index()
    customer_volume = customer_volume.sort_values('orders', ascending=False)
    
    # Top 15 customers
    top_customers = customer_volume.head(15)
    
    fig = go.Figure(data=[go.Bar(
        y=top_customers[customer_col],
        x=top_customers['orders'],
        orientation='h',
        marker_color='#9B59B6',
        text=top_customers['orders'],
        textposition='outside'
    )])
    
    fig.update_layout(
        title={'text': 'Top 15 Customers by Order Volume', 'x': 0.5, 'font': {'size': 20}},
        xaxis_title='Number of Orders',
        yaxis_title='Customer',
        yaxis={'categoryorder': 'total ascending'},
        height=500,
        margin={'l': 200}
    )
    
    fig.show()
    
    # Customer concentration
    print(f"\n📊 Customer Concentration:")
    print(f"   Top 5 customers: {customer_volume.head(5)['orders'].sum() / total_orders * 100:.1f}% of orders")
    print(f"   Top 10 customers: {customer_volume.head(10)['orders'].sum() / total_orders * 100:.1f}% of orders")

## 7. Volume by Domain

In [ ]:
# Domain breakdown
domain_col = COLUMN_MAP['domain']

if domain_col in analysis_df.columns:
    domain_volume = analysis_df.groupby(domain_col).agg(
        orders=('quantity', 'count'),
        quantity=('quantity', 'sum')
    ).reset_index()
    domain_volume = domain_volume.sort_values('orders', ascending=False)
    
    fig = px.pie(
        domain_volume,
        values='orders',
        names=domain_col,
        title='Order Distribution by Domain',
        hole=0.3
    )
    
    fig.update_traces(textposition='outside', textinfo='percent+label')
    fig.update_layout(height=450)
    fig.show()

In [ ]:
# Domain → Product hierarchy treemap
if domain_col in analysis_df.columns and product_col in analysis_df.columns:
    hierarchy_df = analysis_df.groupby([domain_col, product_col]).size().reset_index(name='orders')
    
    # Filter to visible portions
    hierarchy_df = hierarchy_df[hierarchy_df['orders'] >= 5]
    
    if len(hierarchy_df) > 0:
        fig = px.treemap(
            hierarchy_df,
            path=[domain_col, product_col],
            values='orders',
            color='orders',
            color_continuous_scale='Viridis',
            title='Volume Distribution: Domain → Product Hierarchy'
        )
        
        fig.update_layout(height=600)
        fig.show()

## 8. Volume Trends by Product

In [ ]:
# Stacked area: Volume by product over time
if product_col in analysis_df.columns:
    # Get top 5 products
    top_5_products = product_volume.head(5)[product_col].tolist()
    
    trend_product_df = analysis_df[
        (analysis_df[product_col].isin(top_5_products)) & 
        (analysis_df['added_date'].notna())
    ].copy()
    
    trend_product_df['week_start'] = trend_product_df['added_date'].dt.to_period('W').dt.start_time
    
    weekly_product = trend_product_df.groupby(['week_start', product_col]).size().reset_index(name='orders')
    weekly_product = weekly_product.pivot(index='week_start', columns=product_col, values='orders').fillna(0)
    
    # Last 12 weeks
    weekly_product = weekly_product.tail(12)
    
    fig = go.Figure()
    colors = px.colors.qualitative.Set2
    
    for i, product in enumerate(top_5_products):
        if product in weekly_product.columns:
            fig.add_trace(go.Scatter(
                x=weekly_product.index,
                y=weekly_product[product],
                name=product[:30] + '...' if len(str(product)) > 30 else product,
                mode='lines',
                stackgroup='one',
                line=dict(width=0.5, color=colors[i % len(colors)])
            ))
    
    fig.update_layout(
        title={'text': 'Weekly Volume Trend by Top 5 Products', 'x': 0.5, 'font': {'size': 20}},
        xaxis_title='Week',
        yaxis_title='Order Count',
        height=450,
        legend={'orientation': 'h', 'y': -0.2}
    )
    
    fig.show()

## 9. Day-of-Week Pattern

In [ ]:
# Order pattern by day of week
dow_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

dow_volume = analysis_df[analysis_df['day_of_week'].notna()].groupby('day_of_week').size().reindex(dow_order).fillna(0)

fig = go.Figure(data=[go.Bar(
    x=dow_volume.index,
    y=dow_volume.values,
    marker_color=['#3498DB', '#3498DB', '#3498DB', '#3498DB', '#3498DB', '#95A5A6', '#95A5A6'],
    text=dow_volume.values.astype(int),
    textposition='outside'
)])

fig.update_layout(
    title={'text': 'Order Volume by Day of Week', 'x': 0.5, 'font': {'size': 20}},
    xaxis_title='Day of Week',
    yaxis_title='Number of Orders',
    height=400
)

fig.show()

## 10. Complete Volume Summary Table

In [ ]:
# Complete summary by product
print("\n📊 COMPLETE VOLUME SUMMARY BY PRODUCT")
print("="*90)

if product_col in analysis_df.columns:
    product_summary = product_volume.copy()
    product_summary['pct_of_total'] = (product_summary['orders'] / total_orders * 100).round(1)
    product_summary['avg_qty'] = (product_summary['quantity'] / product_summary['orders']).round(1)
    product_summary.columns = ['Product', 'Orders', 'Quantity', '% of Total', 'Avg Qty']
    print(product_summary.head(20).to_string(index=False))

## 11. Export Results

In [ ]:
# Export to Excel
export_filename = f"volume_demand_analysis_{datetime.now().strftime('%Y%m%d_%H%M')}.xlsx"

with pd.ExcelWriter(export_filename, engine='openpyxl') as writer:
    # Summary
    summary_data = pd.DataFrame({
        'Metric': ['Total Orders', 'Unique Orders', 'Total Quantity', 'Avg Qty/Order',
                   'Unique Products', 'Unique Customers'],
        'Value': [total_orders, unique_orders, total_quantity, round(avg_qty_per_order, 1),
                  unique_products, unique_customers]
    })
    summary_data.to_excel(writer, sheet_name='Summary', index=False)
    
    # By Product
    if product_col in analysis_df.columns:
        product_volume.to_excel(writer, sheet_name='By Product', index=False)
    
    # By Customer
    if customer_col in analysis_df.columns:
        customer_volume.to_excel(writer, sheet_name='By Customer', index=False)
    
    # Weekly Trend
    weekly_volume.to_excel(writer, sheet_name='Weekly Trend', index=False)
    
    # Monthly Trend
    monthly_volume.to_excel(writer, sheet_name='Monthly Trend', index=False)

print(f"\n✅ Results exported to: {export_filename}")
# files.download() - uncomment if using Colab
# files.download(export_filename)

---

## 📋 Summary

This notebook analyzed order volumes and demand patterns:

| Analysis | Description |
|----------|-------------|
| **Weekly Trend** | Order volume over past 12 weeks |
| **Monthly Growth** | Month-over-month growth rate |
| **Product Mix** | Top products by volume |
| **Customer Concentration** | Top customers and their share |
| **Domain Distribution** | Volume breakdown by domain |
| **Day-of-Week** | Order patterns throughout the week |

### Key Insights
1. Identify high-volume products for resource planning
2. Track growth trends for forecasting
3. Monitor customer concentration risk
4. Plan capacity based on weekly patterns